In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('texts/input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

#Here’s what makes a RNN recurrent: it uses the same weights for each step. More specifically, 
#a typical vanilla RNN uses only 3 sets of weights to perform its calculations:
#Wxh​, used for all x_t​ → h_t​ links.
#Whh​, used for all h_{t-1}​ → h_t​ links.
#Why​, used for all h_t​ → y_t​ links.

Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output


#We’ll also use two biases for our RNN:
#b_h, added when calculating h_t.
#b_y, added when calculating y_t.
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

the format of output required to calculate the loss should be one-hot encoded vectors.
<img src="https://miro.medium.com/max/684/1*JXpSMnwL7Hx1cGny310DYg.png">

In [31]:
# Transform the result in probabilities for the next chars
def softmax(xs):
	return np.exp(xs) / np.sum(np.exp(xs))

In [74]:
#The np.tanh function implements a non-linearity that squashes the activations to the range [-1, 1]
#Notice briefly how this works: There are two terms inside of the tanh: one is based on the previous hidden state and one is based on the current input. 
#In numpy np.dot is matrix multiplication. The two intermediates interact with addition, and then get squashed by the tanh into the new state vector. 
#If you’re more comfortable with math notation, we can also write the hidden state update as ht=tanh(Whhht−1+Wxhxt), where tanh is applied elementwise.
def tanh_activation(input_to_hidden_weight, previous_hidden_state, hidden_to_hidden_weight, current_hidden_state,  bh):
	return np.tanh(np.dot(input_to_hidden_weight, previous_hidden_state) + np.dot(hidden_to_hidden_weight, current_hidden_state) + bh)


In [89]:
def cross_entropy_loss(ps, target):
	return -np.log(ps[target,0])

In [83]:
#The forward pass use the parameters of the model (Wxh, Whh, Why, bh, by) to calculate the next char given a char from the trainning set.
def forward(inputs, targets, hprev):
	xs, hs, ys, ps = {}, {}, {}, {}
	hs[-1] = np.copy(hprev)

	for t in range(len(inputs)):
		#xs[t] is the vector that encode the char at position t 
		xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
		xs[t][inputs[t]] = 1
		activation = tanh_activation(Wxh, xs[t],Whh, hs[t-1], bh) # hidden state
		# taking the dot product between the activation and the weight matrix -- this is called the "net input" to the current layer
		net = np.dot(Why, activation) + by
		prediction = softmax(net) # probabilities for next chars
		hs[t] = activation
		ys[t] = net
		ps[t] = prediction
	return xs, hs, ys, ps


In [94]:
def tanh_deriv(activationFuncResult, dh):
	return (1 - activationFuncResult * activationFuncResult) * dh 

In [107]:
# We can not use simple “derivative”  or slope of the target function at a specific point.
# Because that RNN requires Adaptative Gradients (AdaGrad) - It is technically referred to as a first-order optimization algorithm as 
# it explicitly makes use of the first order derivative of the target objective function.
def backprop(inputs, targets, activations, predictions, xs):
	# backward pass: compute gradients going backwards
	dWxh = np.zeros_like(Wxh)
	dWhh = np.zeros_like(Whh)
	dWhy = np.zeros_like(Why)
	dbh = np.zeros_like(bh)
	dby = np.zeros_like(by)
	dhnext = np.zeros_like(activations[0])

	for t in reversed(range(len(inputs))):
		prediction = np.copy(predictions[t])
		prediction[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
		
		dWhy += np.dot(prediction, activations[t].T)
		dh = np.dot(Why.T, prediction) + dhnext # backprop into h
		direction = tanh_deriv(activations[t], dh) # backprop through tanh nonlinearity
		
		dWxh += np.dot(direction, xs[t].T)
		dWhh += np.dot(direction, activations[t-1].T)
		dhnext = np.dot(Whh.T, direction)
		dby += prediction    
		dbh += direction
    
	for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
		np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    
	return  dWxh, dWhh, dWhy, dbh, dby, activations[len(inputs)-1]


In [104]:
def sample(previous_hidden_state, start_index, text_length):
	""" 
	sample a sequence of integers from the model 
	h is memory state, seed_ix is seed letter for first time step
	"""
	x = np.zeros((vocab_size, 1))
	x[start_index] = 1
	indexes = []
	for t in range(text_length):
		h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, previous_hidden_state) + bh)
		y = np.dot(Why, previous_hidden_state) + by
		p = np.exp(y) / np.sum(np.exp(y))
		index = np.random.choice(range(vocab_size), p=p.ravel())
		x = np.zeros((vocab_size, 1))
		x[index] = 1
		indexes.append(index)
	return indexes


In [61]:
sample_ix = sample(hprev, inputs[0], 200)
''.join(ix_to_char[ix] for ix in sample_ix)

'ntp  htacte   lp Ccwsscgrs  bsd ttie ettisnat  e etsc a et li  ethiepetae  cev tt yu.lityticplia?y  lratitidspe a tsd dse di  yes  bigtsitoyresyab esk !htiRatad rtc rtttbttti csdekepcil te bs s dtwnye'

In [137]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
epochs = 50000000

In [127]:
# perform parameter(weights) update with Adagrad
def update_weights(dWxh, dWhh, dWhy, dbh, dby):
	for weights, dWeights, memWeights in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby], [mWxh, mWhh, mWhy, mbh, mby]):
		memWeights += dWeights * dWeights
		weights += -learning_rate * dWeights / np.sqrt(memWeights + 1e-8) # adagrad update


In [133]:
def calculate_loss(ps, targets):
	loss = 0
	for t in range(len(inputs)):
		loss += cross_entropy_loss(ps[t],targets[t])
        
	return smooth_loss * 0.999 + loss * 0.001

In [138]:
for n in np.arange(0, epochs):
	# prepare inputs (we're sweeping from left to right in steps seq_length long)
	if p+seq_length+1 >= len(data) or n == 0: 
		hprev = np.zeros((hidden_size,1)) # reset RNN memory
		p = 0 # go from start of data

	inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
	targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
	xs, hs, ys, ps = forward(inputs, targets, hprev)
	dWxh, dWhh, dWhy, dbh, dby, hprev = backprop(inputs, targets, hs, ps, xs)    
	smooth_loss = calculate_loss(ps, targets)
	update_weights(dWxh, dWhh, dWhy, dbh, dby)
	p += seq_length # move data pointer
	
	# sample from the model now and then
	if n % 100 == 0:
		sample_ix = sample(hprev, inputs[0], 200)
		txt = ''.join(ix_to_char[ix] for ix in sample_ix)
		print ('----\n %s \n----' % (txt, ))
		print ('iter %d, loss: %f' % (n, smooth_loss))


----
 us ss uu sss  s s  susp suss    s uuss sususus    susu   u  usssu  uu s su  sr  u ssususs s u ss  u  u   su u uss  us us u   sssuur suus  sus s su u ur u  suus  uu ss      u u  ssu  su suu  s  uus s s 
----
iter 0, loss: 98.723237
----
 dnnrrid nrdbnncnnrtclmgnnnnblirlnrluoswnnrndnnrrfrrnnmtnebtitminsnbnhfcannfndmtnrsbitldldebnnrsondnfnbna ncwnpin rlabnfnpm ecdnnlslnbncpn rncvvrrdnndbibli albttvrnanlvdneetrvrlcdrmpnvrlcndndmndsndtnry 
----
iter 100, loss: 101.249942
----
 o      ..e  hl .ee    . . t tl..   e it     .  h .-. .     e    i   o.  e   l    e .   .   . . . t   .  e.   .  oo e    ht  .  .  e   .      . . . .tl   .     .  .  r        .   h . .  eh    . t   t   
----
iter 200, loss: 99.198765
----
  u ,y uaa   i ute    ai  ,eh    okhi   ,    hu ,a l ilh i,  r     ha  au   ,yu .yal a u as .   ly  hh lo   u.h  ,eiuoa o el r   u e   lu     y erl yy, s . -ue ruo oy c,       h   h r uuyu ,u ou ue,ieg 
----
iter 300, loss: 96.705266
----
 oe ccnstdrannrnasao npscg ysglnsmknrngd

----
 UpAplpppphhRlUUppppUppUlhhUhhAhppUpUpphhlpippppplpUpppplhUhhplpRhRphhhlARhlhApylphpUBRApUphpUlplpppppUplphippAUUhhUlpphUhpURpphppppUpphhRplpphpUUUhpUhhUhphhpppAhpUUppUphUUhphppRlhphlhphhlhphwRhphlhhpp 
----
iter 3500, loss: 61.693794
----
 uciacimemwvsctiavemtcaditwmmParacicpudsaiwtmttmmtttctbuytyahgncyhovhhaatbaaaauamairjctopaaioctjoocayycwcmowtiwdsBilcddctstayvsBahntumiuuedtbtiuatctaaattmcaymhdisTebcaayiwumosufciot,plcdmpwtoiyiarmmomh 
----
iter 3600, loss: 61.482169
----
 jPPPPI
EPPPPPPPPIPPPtPP
jPPPPP
LPPPPP
PSP
PPPP
PP
P
PP
P

P
P
P

PPPP


PPP
I
j
PIPPPPPP

IPP

PPIPPPPIP
PPj

PP


I
IIjPPPP
PIP

jP
PPI
PPPPPP
PPP
jPjPI
P
IPP
PPP
P
PPP
PP

PPP
PPPjPPPP

PPP
P
PPPPPP 
----
iter 3700, loss: 61.243418
----
 tddoyacomdhdtcrctmowocmttoyuwadcuocdwanccwncocPopceouytoitarywdctatBswemcaoadfSwttdtcglaurlwbudnycoacwwogauabdedwcowhjra-moncuctogorhmcwocPwtathrtaatittmhmwwwytwiocwyhaPctiowatckcimdtccPddmcdiwwetdadw 
----
iter 3800, loss: 60.958911
----
 rttpdrpchgdgnpctcnert ndddrtpcrvm 

----
 ervnvrnlernrneaerervriornlerrkorfnnxerfvvvffuiveofnvenlrvnforrrbrfler fuerarlyffvekrvrfxffvrfnrtolb fsinrxoevvninviofrttonroarrfrrbffrrxrinrdfovrvalbendrrarriprbnrrrroyvrr uolfrarcrnxifraorrmvrxae,ooe 
----
iter 7000, loss: 57.575298
----
  e ,gtt teiihee,h  eeyoeye ! leea ete,ilo,e kitcoelc sd eol.e. toie,ees teo gtr   tyho.o 'iree iee e t  ou,s h, eu i.s iuesui?se!e yhrswy'ghslhorlyees   totetgdsg ligetrhi ee.,atl eeyt .o sfc,lee  i   
----
iter 7100, loss: 57.470184
----
 rrr nrrwnuudnwunfnm fmrunnmfun vnmtnwfrwrmrnnnunnnlmgurwnnfmwrnrnrrulnsvnfnrwnlt brndnnudmnnrmrnrndrdubnnnnnufrnuqru rfwnn nnburfnnrufnfuntnmrnwrrnbfuwhnrnrrdnrudbnrrrunmtsrubnrbnu nurrgnnugcunrnnwffm 
----
iter 7200, loss: 57.391544
----
  ooooo ooooeo  oo    oooo  o oo,ooo oo   o lo oooooo  ooolSoo ooo o ooo  ooo ooo ooooo        oooo o o
 o ooo Soooo oooo oo o o
 oo  ooooo ooo o ooo      ooo ooo   ooo oo  o oooo  oool  o o  oooooooo  
----
iter 7300, loss: 57.281160
----
 tdntvvnn cttnnhnvngrftlddtttcsndtd

----
 ec  t ette ttitei t    te ts e   ee    iceete  eceeet   ttcete ei titgrt   teetsstt  tte et  e eesistet   ticit
i. iiet eeeti  eeiiiicss e istt e     egtttte   ittesetettt  s eegtt iieti e  ti e ist e 
----
iter 10500, loss: 56.049889
----
 enatanmsmsctsfnttncmeammoPoesttorccmttsaaapwabtteowscltttbawintmcucmtpyatsoootttmutstatmssndcrceweentctdmcisichtslsaciohtcttcsmtascupmnctwttcvopdowstyutmbitlaohnaanutoiohmrdttaitIyaakyeiettoeewtagsdtm 
----
iter 10600, loss: 55.936548
----
 l iiiiiiie i tliihh seiet  yl.   igi ii hie.i   iy i  i slieii i iet     yi s i R aisi. i  s   e l elhh siei  iie   ie tii    ii,e  i titik  eiiii     i  ii ilost   t  i  ril iii '  i l.i l iicii. i e 
----
iter 10700, loss: 55.967506
----
 clpv grstntc s gptnxnra bnttd stmrcslngsnndccssncnrst?nntnnisesgdsbscdtntcs ccdgtdeenxcnn cltnlsiptdn-nccnmducncan t,sdt ssmlnvnddsxc nldtgllssvsnsr estcn nttentmgsrttncrrgngctncsetltxclfttc sntrrxtbl 
----
iter 10800, loss: 56.004082
----
 ey ayoeatioli  e  ls pte    e 

----
 rwfufndnrdnfurnrnfrnnfnuunrnfnrnnsrdnnvmwsnmnn nnrnnnfdsundgnnnrnn rnnnfnrndrrfnnndnnnnmmfnnbnntrnnmfnnnrnnqnnvnnrnrnrurufnvnnnrnnumvrnnnnffwdmnmnnuwlnmfrnrrvnnrtrmndnmumnmnmfuncrnnnnrfnrnfnuwfunmdnun 
----
iter 13900, loss: 55.053343
----
  nr ntrgnnlnlnnrr e nips  lmc vnnenmp mrn svnhnnt rn nnn ntc ssnefnngrdrg.tcm  rnnnnttclncnnnnnsd ccnnnncgnnn ngn rgl nfntnnnnitnsugwntgnnrn  rr trmnvctrnrnndnnnnnnnnccrnw s csrndntsrnd nn rdarrrtrnnn 
----
iter 14000, loss: 55.023641
----
 hij    sli itt. t sii     st ei  eittsiissie eost yii   i   re t l iii siy s  i ii  iir  i c    o s t  t  siiiiie ei  t.it ie otis    ti  tt iet ie e.i tpl  e    sise  o  di.   ii  ei sio  i    i  tik 
----
iter 14100, loss: 55.088495
----
  ts  st oees?eust  llie  ei tah teel osc e uto  t a sehcse gt ; tia  helk ii ro or  ltr sidg cil tri tlietioi  gtlh!aesou,,tap . stsys.hue t e or shgmiisccti sellr udaiao.r iktrelitt ml   lte't   hk o 
----
iter 14200, loss: 55.103232
----
  esasegs ,.kh e.esre;  ee po e

...

----
 ncnmsdmgnsscnctcnnrnsasndrnnvsncsc tpntptrnnd sscctccnnncdqnnn,nvtyvcdpcddcns dctsclnsn vtdnncntccdnpnntgnnmlnnyctntdvnnsnrncsntfcntnnnm nsdgngtdnnscpccnrcssnncnsndnsangsssgrnlmvdffgsnbtcbnctncdstsscn 
----
iter 309700, loss: 41.097732
----
 lllllllllllllllllllljllllllllllllljllllllllllllllllllllljlljlllllllllllllllllllljlljlllllljllllllllljlllllllllllllllllllljlllllllllllllullllllllllllljlllllllllllllllllllllllllllllllllllllllljlllllllll 
----
iter 309800, loss: 41.095942
----
 .e     a se  e  .  s ss  e  s  ss s  ee    
ase. s    o    see e .  e  s.   es e e  a       e. s ee .  e    s o     e eee  .e  . ahh e e hs s   !  s!sss  es  . ae s se.   ae. s  s   si   . e  seh    s 
----
iter 309900, loss: 40.999354
----
 ccccecicwctmbaeacscctomatmtaoatruattabwfcitiotwekcvdwthwicwnsbitgeoiwccpiosatsyabwtfwtPBuytiiotwcrictitwtdtctwtcopaancibfedctaiidhitmddtmwdPaswcmcmiatacpsuawobwawhtaiiBmutarcaaipbmccoairifpccyatodiasd 
----
iter 310000, loss: 41.064631
----
 gefi eedegeeeeee    gfst ?

----
 ohlsyatsel yesloolsllo,oolisyltsRloolleololaooelllryslseosoRro olselotiooossooosehheootlaeoohhleoooololoooyooiolg eolltwtttlreurRoerosoioyossohtoltsllRtoloeopoooohhellllhlRheltoRlhlol o.oRoo lttlroteo 
----
iter 313100, loss: 41.054294
----
      e                            hhp           h   h a     h               a h hh   e                       h  a     e  h                         '  a a      o   hh       h             a  h           
----
iter 313200, loss: 40.972243
----
 hoahohohohhhhohhhhhhhhhhhhhhhhheohohohhhishohhhooosha hhohohhhohhhhhhooohhhohhhohohhhhoohhhoohTshohhhhohhuoohhhhhhhohhhhhhhohuhheohhhhhhohhhhhohhohhuohhehhhhhhhhehhhohhhhohhhohhhhohhhhhh hhhohoohhhehh 
----
iter 313300, loss: 41.014441
----
                                                                                                                                                                                                          
----
iter 313400, loss: 40.961541
----
 rrllll   erllll  rlllor  l

----
 eeeaRceooayoieeieiutlotuyyetieiayieytyrieyiyogyaceooaylyeytiayoeieiylatty,ieyyruisaeeeaioyieoetiysieyeigaeyie tieyehyeciitaaiiiyeioieisosoaysaeayaesiieayeitiiyoueeilioaeasoiyeeeeeiiaoaesioaoe'ioaiyiet 
----
iter 316500, loss: 40.989098
----
 oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooiooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooiooooo 
----
iter 316600, loss: 40.967630
----
 msrluol rusururnhuuhmuf srwmu mcruhuruu ouu um rluuhu mo r uum dufrf  rs  sm ruurmmru tmmmum  ur mumuummmumsmumro uspmmrrvurr vm mltrouh uro ruljwurfhr g fuuwtuw uupu   g uururm rmumuhurourl.  um moo  
----
iter 316700, loss: 40.902141
----
 aroooooohooolharorohloelooroarooaoolooooooooouooaaoohoooooooooooooaoooloaooaooomaaoaoaooooaahhomooooolhouaooaaoolhaooooohoemoooaomoooo hoelohooaoaiooloooaoioaoohooaooralooaooooaaaroooamooooorooaoooooh 
----
iter 316800, loss: 40.881518
----
 bdmnmdudmmdnsdmdwmnnnnnwcm

----
 ddgdggddddddddddddddeddddgddddgddddddddddgdddddddddddggddgdgdgdddfdddddddddddfdddegddgdddddddddddgdgdddddddddgdddddddgdddddddddddddddddddddddddedgdddgddddddddgddgdddddddgddddddddddddddddgddddddgdddddd 
----
iter 319900, loss: 40.927228
----
 n unacstnnnrnsnvnpfd nnnminnnnnnndmdmvstcdnntttnsnsncgpnrmnnrentnnndmn nrnnttnnnnqnongqnnnncntttnnntdnnmrcndt sfvcnnnsnnnntnnnftafnnftmmgttytnnrnnzn ncntbamtntn atntasnt nv nncnmtgntgnwmgngns.nnntn na 
----
iter 320000, loss: 40.932406
----
 eeeeeeeeeeeeeet eeeeeeeeeeeeeeeeeeeeeeeeeeeaeeeeeeeeeeeeeeeeeeeeeeeeefeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeleeeeeeeeeeeeeeeeeeeleeeeeeeeeeeeeeaaeeeeeeeaeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee 
----
iter 320100, loss: 40.831653
----
 eeieieeeeieeeeeeieeeiee eeeeiieeieeeeeeeeeeeeieeoeieeieeeeeeieiieeieeieeeeeeeeeeeeeeeieeieeeeeieeeieeseei eeeieeeeeeieeeeeeeeeeeioeeeieoeeiieeieeieeeieeieeeeeeeeeieeeeeeeieeeeeeeeeeeeeeeieie.eeeeeeeie 
----
iter 320200, loss: 40.928211
----
 eieeeeeieeieefeeeaaaeaeeee

KeyboardInterrupt: 